Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices
import numpy as np
import numpy.random as rand
from copy import deepcopy
from tqdm import tqdm

import lab9_lib

In [10]:
# Turn this back to 1000 to solve the real problem
INDIVIDUAL_SIZE = 30

In [3]:
fitness = lab9_lib.make_problem(1)
for n in range(2**10):
    ind = [int(b) for b in str('{0:010b}'.format(n))]
    if fitness(ind) == 1 :
        print(ind)

print(fitness.calls)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1024


In [4]:
class Individual :
    def __init__(self, fitness_func, genotype = None) -> None:
        if genotype is None :
            self.genotype = np.array([rand.choice([0,1]) for _ in range(INDIVIDUAL_SIZE)])
        else :
            self.genotype = genotype
        self.fitness = fitness_func(self.genotype)

    # Overload [] method
    def __getitem__(self, key) :
        return self.genotype[key]
    
    # Overload [] assignment method
    def __setitem__(self, key, newvalue) :
        self.genotype[key] = newvalue

In [12]:
class Population :
    def __init__(self, size, offspring_size, fitness) -> None:
        self.size = size
        self.offspring_size = offspring_size
        self.individuals = [Individual(fitness) for _ in range(self.size)]
        self.fitness = fitness
        self.cnt_gen = 0

    # Allows to iterate directly over individuals
    def __iter__(self) :
        return self.individuals.__iter__()

    # Mutates one random gene of an individual
    def mutate(self, individual : Individual) :
        # Mutation
        new_ind = deepcopy(individual)
        loci = rand.randint(0, INDIVIDUAL_SIZE)
        new_ind[loci] = not new_ind[loci]
        
        # Fitness update
        new_ind.fitness = self.fitness(new_ind.genotype)
            
        return new_ind
    
    # Recombines two parents using uniform crossover (or a computationally lighter approach in which each individual get exactly 50% genes randomly from each parent)
    def crossover(self, ind1 : Individual, ind2 : Individual) :
        new_genotype = np.zeros(INDIVIDUAL_SIZE)
        indices = np.arange(INDIVIDUAL_SIZE)
        rand.shuffle(indices)
        limit = int(np.floor(INDIVIDUAL_SIZE / 2))
        new_genotype[indices[:limit]] = ind1[indices[:limit]]
        new_genotype[indices[limit:]] = ind2[indices[limit:]]
        new_ind = Individual(self.fitness, new_genotype)
        # new_ind = Individual(self.fitness, [rand.choice([ind1[i], ind2[i]]) for i in range(INDIVIDUAL_SIZE)])
        return new_ind

    # Perform parent selection using tournament selection
    def parent_selection(self, parent_number, tournament_size) :
        parents = []
        for _ in range(parent_number) :
            candidates = rand.choice(self.individuals, size=tournament_size)
            parent = max(candidates, key=lambda i : i.fitness)
            parents.append(parent)

        return parents
    
    # Create offspring_size new individuals, either by mutation or by crossover
    def create_offspring(self, parents, mutation_prob) :
        new_individuals = []
        for _ in range(self.offspring_size) :
            if rand.rand() < mutation_prob :
                parent = rand.choice(parents)
                new_individuals.append(self.mutate(parent))
            else :
                par1, par2 = rand.choice(parents, size=2)
                new_individuals.append(self.crossover(par1, par2))
        
        return new_individuals
    
    # Determine the survivors of the population based on their fitness
    def survivor_selection(self, offspring) :
        total_population = self.individuals
        total_population.extend(offspring)
        # Sort all individuals by descending fitness, and keep the fittest one to conserve population size
        surivors = sorted(total_population, key = lambda i : i.fitness, reverse=True)[:self.size]
        return surivors

    # Complete process of a generation
    def evolve(self, parent_number = None, tournament_size = None, mutation_prob = 0.1) :
        if parent_number == None :
            parent_number = int(np.ceil(self.size / 5))
        if tournament_size == None :
            tournament_size = int(np.ceil(parent_number / 2))
        parents = self.parent_selection(parent_number, tournament_size)
        offspring = self.create_offspring(parents, mutation_prob)
        self.individuals = self.survivor_selection(offspring)
        self.cnt_gen += 1


In [55]:
fitness = lab9_lib.make_problem(1)
test = Population(100,50, fitness)
print("Initial population fitness :", np.mean([ind.fitness for ind in test]))
print('----------')
while not np.any(np.array([ind.fitness for ind in test]) == 1) :
    test.evolve()
print(f"Optimal individual found in {test.cnt_gen} generations, with {fitness.calls} calls to fitness function")

Initial population fitness : 0.518
----------
Optimal individual found in 5 generations, with 350 calls to fitness function


In [53]:
# Promote diversity with islands partition
class IslandPopulation(Population) :
    def __init__(self, size, offspring_size, fitness, island_number, migration_fqcy, migration_prob) -> None:
        super().__init__(size, offspring_size, fitness)
        if island_number > size / 4 :
            self.island_number = self.size / 4
        else :
            self.island_number = island_number
        self.migration_fqcy = migration_fqcy
        self.migration_prob = migration_prob
        self.ind_per_island = np.split(np.array(self.individuals), self.island_number)
    
    # Parent selection has to be done in each island
    def parent_selection(self, parent_number, tournament_size, island) :
        parents = []
        island_individuals = self.ind_per_island[island]
        for _ in range(parent_number) :
            candidates = rand.choice(island_individuals, size=tournament_size)
            parent = max(candidates, key=lambda i : i.fitness)
            parents.append(parent)

        return parents    
    
    # Survivor selection is also done locally on each island
    def survivor_selection(self, offspring, island) :
        total_population = self.ind_per_island[island].tolist()
        total_population.extend(offspring)
        # Sort all individuals by descending fitness, and keep the fittest one to conserve population size
        surivors = sorted(total_population, key = lambda i : i.fitness, reverse=True)[:self.size]
        return np.array(surivors)
    
    # Evolve now includes migration between islands
    def evolve(self, parent_number = None, tournament_size = None, mutation_prob = 0.1) :
        if parent_number == None :
            parent_number = int(np.ceil(self.size / (self.island_number * 5)))
        if tournament_size == None :
            tournament_size = int(np.ceil(parent_number / 2))

        if self.cnt_gen % self.migration_fqcy == 0 :
            for island, inds in enumerate(self.ind_per_island) :
                for ind in inds :
                    if rand.rand() > self.migration_prob :
                        continue
                    new_island = rand.choice(np.setdiff1d(np.arange(0, self.island_number), np.array([island])))
                    ind_list = self.ind_per_island[island].tolist()
                    ind_list.remove(ind)
                    self.ind_per_island[island] = np.array(ind_list)
                    self.ind_per_island[new_island] = np.append(self.ind_per_island[new_island], ind)
                    
        for island in range(self.island_number) :
            parents = self.parent_selection(parent_number, tournament_size, island)
            offspring = self.create_offspring(parents, mutation_prob)
            self.ind_per_island[island] = self.survivor_selection(offspring, island)
    
        self.individuals = [ind for island_inds in self.ind_per_island for ind in island_inds]
        self.cnt_gen += 1



In [79]:
fitness = lab9_lib.make_problem(1)
island_pop = IslandPopulation(100,50, fitness, 2, 5, 0.8)
print("Initial population fitness :", np.mean([ind.fitness for ind in island_pop]))
print('----------')
while not np.any(np.array([ind.fitness for ind in island_pop]) == 1) :
    island_pop.evolve()
print(f"Optimal individual found in {island_pop.cnt_gen} generations, with {fitness.calls} calls to fitness function")

Initial population fitness : 0.48699999999999993
----------
Optimal individual found in 9 generations, with 1000 calls to fitness function


In [24]:
np.split(np.array([1,2,3]), 3)

[array([1]), array([2]), array([3])]